In [2]:
import struct
import numpy as np
import intvalpy as ip
import numpy as np
import readfiles

In [3]:
filepath_y = "0.225_lvl_side_a_fast_data.bin"
filepath_x = "-0.205_lvl_side_a_fast_data.bin"

In [4]:
rad = 1/(2**14)

X_data_origin = readfiles.read_files(filepath_x)
Y_data_origin = readfiles.read_files(filepath_y)


In [8]:
def get_avg(data):
    avg = [[0]*8]*1024
    for i in range(len(data)): # 100
        avg = np.add(avg, data[i])
    return np.divide(avg, len(data))

In [9]:
x_avg = get_avg(X_data_origin)
y_avg = get_avg(Y_data_origin)

In [12]:
import matplotlib.pyplot as plt

In [37]:
def scalar_to_interval(x, rad):
    return ip.Interval(x-rad, x+rad)

In [55]:
X = np.vectorize(scalar_to_interval)(x_avg, rad).flatten()
Y = np.vectorize(scalar_to_interval)(y_avg, rad).flatten()


In [53]:
# bounds
a_min = float(np.min(Y).a) - float(np.max(X).b)
a_max = float(np.max(Y).b) - float(np.min(X).a)


t_min = float(np.min(Y).a) / float(np.max(X).b) 
t_max = float(np.max(Y).b) / float(np.min(X).a)

In [52]:
def coeff_Jacarta(X,Y):
    

0.3216912841796875

In [78]:
def jaccart_interval(x, y):
    
    a1, b1 = float(x.a), float(x.b)
    a2, b2 = float(y.a), float(y.b)   
    return (min(b1, b2) - max(a1, a2)) / (max(b1, b2) - min(a1, a2))

def jaccart_intervals(X):
    lowers = [float(x.a) for x in X]
    uppers = [float(x.b) for x in X]
    return (min(uppers) - max(lowers)) / (max(uppers) - min(lowers))

def jaccart_sets(X, Y):
    coeffs = [jaccart_interval(x, y) for x, y in zip(X, Y)]
    return np.array(coeffs)

def coeff_Jaccart(X, Y):    
    if isinstance(X, ip.ClassicalArithmetic) and isinstance(Y, ip.ClassicalArithmetic):
        return jaccart_interval(X, Y)
    return jaccart_sets(X, Y)



In [80]:
a_interval = np.linspace(a_min, a_max, 100)

In [81]:
for a in a_interval:
    print(a)
    print(np.mean(coeff_Jaccart(X+a, Y)))

0.3216912841796875
-0.9887447226519557
0.3221843249388415
-0.9884773505204323
0.3226773656979956
-0.9881965727235975
0.3231704064571496
-0.987901274681794
0.3236634472163037
-0.9875901714033901
0.3241564879754577
-0.9872617320772161
0.3246495287346117
-0.986914009694791
0.3251425694937658
-0.9865441469269323
0.3256356102529198
-0.9861463170405173
0.3261286510120739
-0.9856943964048646
0.3266216917712279
-0.9852484071645058
0.3271147325303819
-0.984833965714316
0.327607773289536
-0.9843483355132203
0.32810081404869
-0.9838143451551775
0.3285938548078441
-0.9832061352015072
0.3290868955669981
-0.9825969615274761
0.3295799363261521
-0.981986677826475
0.3300729770853062
-0.9812843286598687
0.3305660178444602
-0.9805034053603665
0.3310590586036143
-0.9795691392472032
0.3315520993627683
-0.9787333112247845
0.3320451401219223
-0.9777463920155279
0.3325381808810764
-0.9766282349291779
0.3330312216402304
-0.9753642250609988
0.3335242623993845
-0.9738403141787438
0.3340173031585385
-0.9723508982